In [3]:
import sys
from collections import defaultdict
import math,random,re

"""
IBM Model 1: estimate t parameters
EM Algorithm: finding the alignments
"""



class IBM_Model_1(object):
    """
    Stores counts for n-grams and emissions. 
    """

    def __init__(self, n=3):
        #sentences <--bad solution
        self.source = list()
        self.target = list()
        
        self.source_corpos = defaultdict(int)
        self.target_corpos = defaultdict(int)
        #counts
        self.source_target_occurance = defaultdict(int) # c(e,f)
        self.source_occurance = defaultdict(int) # c (e)
        self.conditioned_occurance = defaultdict(int) # c(j|i,l,m)
        self.joint_occurance = defaultdict(int) # c (i,l,m)
        #term-frenquency?
        self.t = defaultdict(float) # t (f|e)
        self.q = defaultdict(float)
        #delta
        self.delta = defaultdict(float)

    def counter(self,source_file,target_file):
        for line in source_file:
            line = line.strip().split(' ')
            
            for word in line:
                self.source_corpos[word] +=1
            # save only words and digis
            line = [x for x in line if re.match("^[A-Za-z0-9_-]*$", x)]
            line.insert(0,'NULL')
            self.source.append(line)
        for line in target_file:
            line = line.strip().split(' ')
            line = [x for x in line if re.match("^[A-Za-z0-9_-]*$", x)]
            self.target.append(line)
            for word in line:
                self.target_corpos[word] +=1
        print ('Load done!')
        print ('Found %i words in source'%len(self.source_corpos))
        print ('Found %i words in target'%len(self.target_corpos))
        print (len(self.source))
        print (len(self.target))

    #def sentences(self,source_file,target_file):
        #load in a pair of sentences at one time
    
    def em(self):
        
        # all counts are zeros (down by defaultdict), t (f|e ) are random values range in [.0,1.]
        for k in range(1, len(self.source)):
            for i in range (0, len(self.target[k])):
                for j in range (0,len(self.source[k])):
                    self.t[tuple((self.target[k][i],self.source[k][j]))] = random.random()
                    self.q[tuple((j,i,len(self.source[k]),len(self.target[k])))] = random.random()
    
    
        # normally should be about 10-20
        for s in range(1,11):
            mk=0
            lk=0
            # iterate every sentence pairs
            for k in range(1, len(self.source)):
                mk = len(self.target[k])
                lk = len(self.source[k])
                for i in range (0, mk):
                    down = 0.0
                    for j in range (0,lk):
                        # 分母
                        down += (self.t[tuple((self.target[k][i],self.source[k][j]))] * self.q[tuple((j,i,lk,mk))])
                        
                    for j in range (1,len(self.source[k])):
                        if down == 0.0:
                            self.delta [tuple((k,i,j))] = 0.0
                        else:
                            self.delta [tuple((k,i,j))] = self.t[tuple((self.target[k][i],self.source[k][j]))] * self.q[tuple((j,i,lk,mk))] /down
                        
                        self.source_target_occurance [tuple((self.source[k][j],self.target[k][i]))] += self.delta [tuple((k,i,j))]
                        self.source_occurance [self.source[k][j]] += self.delta [tuple((k,i,j))]
                        self.conditioned_occurance [tuple((j,i,lk,mk))] += self.delta [tuple((k,i,j))]
                        self.joint_occurance [tuple((i,lk,mk))] += self.delta [tuple((k,i,j))]
            
            # Update t
            sums = 0
            for k,v in self.t.items():
                if self.source_target_occurance [k] * self.source_occurance[k[1]] >0 : 
                    result = self.source_target_occurance [k]/self.source_occurance[k[1]]
                    sums +=1
                    
                else: 
                    result = 0.0
                self.t[k] = result
            
            sumss = 0
            for k,v in self.q.items():
                if self.conditioned_occurance [k] * self.joint_occurance [tuple((k[0],k[1],k[2]))] >0 : 
                    result = self.conditioned_occurance [k] * self.joint_occurance [tuple((k[0],k[1],k[2]))]
                    sumss +=1
                else: 
                    result = 0.0
                self.q[k] = result
            
            print (sumss)
            
            print ('Now..',s)
       
                    
    def write_t (self, output):
         
        for k,v in self.t.items():
            if v >0:
                output.write('%s %s %f\n'%(k[0],k[1],v))
        for k,v in self.q.items():
            if v >0:
                #output.write('%s %s %f\n'%(k[0],k[1],v))
                print (k,v)
    
        output.close()
        print ('Done!')

def usage():
    print ("""
    
        Read in both English and Spanish files
    """)

if __name__ == "__main__":

    
    source_file = open (r'small_corpus.en','r')
    target_file = open (r'small_corpus.es','r')
    output_file = open ('small_output','w')
    # Initialize a trigram counter
    my_model = IBM_Model_1()
    my_model.counter(source_file,target_file)
    my_model.em()
    my_model.write_t(output_file)
    
    

Load done!
Found 756 words in source
Found 696 words in target
100
100
409
Now.. 1
409
Now.. 2
409
Now.. 3
409
Now.. 4
409
Now.. 5
409
Now.. 6
409
Now.. 7
409
Now.. 8
409
Now.. 9
409
Now.. 10
(11, 8, 14, 12) 0.08642838489995583
(6, 13, 8, 15) 0.013770100894998848
(6, 8, 14, 12) 0.0020951362491270203
(7, 8, 14, 12) 0.10804753051648788
(44, 40, 46, 43) 0.011658446596375176
(13, 28, 26, 29) 0.08186202059820985
(4, 8, 15, 14) 0.005946691688121292
(5, 8, 14, 16) 0.08749544657106739
(10, 14, 16, 17) 0.008039347182138409
(1, 19, 22, 21) 0.004865064063569816
(14, 15, 17, 19) 0.070129996258404
(8, 26, 29, 30) 0.03657885645961569
(10, 17, 19, 22) 0.05998022912634287
(11, 15, 14, 16) 0.07896343754603437
(42, 40, 46, 43) 0.03274115565417137
(8, 15, 17, 19) 0.07550698516178407
(35, 40, 46, 43) 0.058248918714934844
(8, 8, 15, 14) 0.017680235086697787
(18, 19, 22, 21) 0.02121297532791469
(27, 40, 46, 43) 0.031813610344226306
(10, 15, 14, 16) 0.14708480071645255
(16, 16, 17, 19) 0.028220844705742067
(